In [1]:
import os
import tqdm
import warnings
warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore', category=UserWarning)

saving_root = './recode'

# --  softs
## 'spanve-k','spanve-d', 'p.spanve-k','p.spanve-d'
## 'somde','moran','geary', 'spatialde',
## 'r-sparkx','r-gitto-rank','r-gitto-sirank', 'r-meringue',
## 'sepal', 'seurat','cell_ranger','seurat_v3'

def running_experiment(
    softs = ['seurat','cell_ranger','seurat_v3'],
    datasets_id = 'SpatialBench', # '10XGenomics' ; folder under data
    filter_code = '',
    filter_datasets_flag = False,
    check_runned = False,
):
    
    def filter_datasets(name,max_val=80,min_val=0):
        if not filter_datasets_flag: 
            if filter_code in name:
                return True
            else: return False
        if int(name.split('_')[1]) > max_val:
            return False
        if int(name.split('_')[1]) <= min_val:
            return False
        else :
            print(name)
            return True
        
    base_dir = f'./data/{datasets_id}'
    dataset_dirs = [os.path.join(base_dir,i) for i in os.listdir(base_dir) if (filter_datasets(i) and i.endswith('h5ad'))]
    bar = tqdm.tqdm(total=len(dataset_dirs)*len(softs))

    for dataset_dir in dataset_dirs:
        if not dataset_dir.endswith('h5ad'): 
            continue
        for soft in softs:
            bar.set_description(f'{dataset_dir}_{soft}')
            dataset_name = os.path.basename(dataset_dir)
            saving_path = os.path.join(saving_root,datasets_id,dataset_name,soft)
            os.makedirs(saving_path, exist_ok=True)
            if check_runned and any([i.endswith('.csv') for i in os.listdir(saving_path)]):
                bar.update(1)
                continue

            memory_info = os.path.join(saving_path,'memory_info.txt')
            command = f'\\time --output {memory_info} python ./utils/frame.py --dataset {dataset_dir} --soft {soft} --saving_path {saving_path}'
            try:
                os.system(command)
            except:
                print(command)
                !rm -rf $saving_path
                raise Exception(f'error in {dataset_dir} {soft}')
                os.exit(1)
            bar.update(1)
    bar.close()

In [4]:
softs = ['spanve-k','spanve-d', #'p.spanve-k','p.spanve-d', # preprocess not used in simulation.
         'somde','moran','geary', 'spatialde',
         'r-sparkx','r-gitto-rank','r-gitto-sirank', 'r-meringue',
         'sepal', 'cell_ranger', 
         'r-heartsvg', 'r-spagene', 'scgco',
        ]